### Q1 output

In [33]:
import requests
import re
import pandas as pd

requests.packages.urllib3.disable_warnings()

url='https://www.imdb.com/chart/top/'
headers={
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
}

txt= requests.get(url, headers=headers, verify=False).text.replace("\xa0"," ").replace("&#x27;","\'")
name=re.findall(r'<h3 class="ipc-title__text">\d{1,3}. (.+?)</h3>',txt)

yearandlength=re.findall(r'<span class=".+?cli-title-metadata-item">(.+?)</span>',txt)
year, length=[],[]
for i in yearandlength:
    if re.search('\d{4}',i):
        year.append(i)
    if (re.search('\d+h \d+m',i)) or (re.search('\d+h',i)) or (re.search('\d+m',i)):
        length.append(i)

rating=re.findall(r'"IMDb rating: ([\d.]+?)"',txt)
vote=re.findall(r'<span class="ipc-rating-star--voteCount"> \(<!-- -->(.+?)<!-- -->\)</span>', txt)

q1df=pd.DataFrame({"name":name,"year":year, "rating":rating, "vote":vote, "length":length})
mask = q1df['vote'].str.endswith("M")
q1df['vote2']=q1df[mask]['vote'].str.strip("M")
q1df.loc[~mask,['vote2']]=q1df[~mask]['vote'].str.strip("K")
q1df['vote2']=q1df['vote2'].astype('float')
q1df.loc[mask,['vote2']]=q1df.loc[mask,['vote2']]*1000000
q1df.loc[~mask,['vote2']]=q1df.loc[~mask,['vote2']]*1000

q1df['vote2']=q1df['vote2'].astype('int')
q1df['name']=q1df['name'].astype('str')
q1df['year']=q1df['year'].astype('int')
q1df['rating']=q1df['rating'].astype('float')
q1df['vote']=q1df['vote'].astype('str')
q1df['length']=q1df['length'].astype('str')

q1sample=q1df.loc[q1df['year']>=2020].groupby('year').agg({'rating':'mean','vote2':'mean'})
q1sample.columns=['Ave_rating','Ave_vote']
q1sample=q1sample.round({'Ave_rating':1})
q1sample['Ave_vote']=q1sample['Ave_vote'].astype('int')

q1df.to_csv('./Q1_movies.csv', index=False)

In [35]:
q1df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    250 non-null    object 
 1   year    250 non-null    int32  
 2   rating  250 non-null    float64
 3   vote    250 non-null    object 
 4   length  250 non-null    object 
 5   vote2   250 non-null    int32  
dtypes: float64(1), int32(2), object(3)
memory usage: 9.9+ KB


In [36]:
q1df.head(3)

,name,year,rating,vote,length,vote2
0,The Shawshank Redemption,1994,9.3,2.8M,2h 22m,2800000
1,The Godfather,1972,9.2,2M,2h 55m,2000000
2,The Dark Knight,2008,9.0,2.8M,2h 32m,2800000


In [37]:
q1df.tail(3)

,name,year,rating,vote,length,vote2
247,Life of Brian,1979,8.0,415K,1h 34m,415000
248,Aladdin,1992,8.0,449K,1h 30m,449000
249,Drishyam,2015,8.2,91K,2h 43m,91000


In [38]:
q1sample

,Ave_rating,Ave_vote
year,,
2020,8.2,142000
2021,8.5,522000
2022,8.3,632000
2023,8.6,357000


### Q2 output

In [42]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
requests.packages.urllib3.disable_warnings()

url3='https://realpython.github.io/fake-jobs/'
headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'}
jobs= BeautifulSoup(requests.get(url3, headers=headers, verify=False).text, 'html.parser')

position, company, city, state=[],[],[],[]
for i in jobs.find_all('h2'):
    position.append(i.text)
for i in jobs.find_all('h3'):
    company.append(i.text)
for i in jobs.find_all('p', {'class':'location'}):
    j,k = list(re.findall('(\w+), (\w+)',i.text)[0])
    city.append(j)
    state.append(k)

q2df=pd.DataFrame({'position':position,'company':company,'city':city, 'state':state})

q2sample=q2df.loc[(q2df['state']=='AA')&(q2df['position'].str.contains(re.compile('[eE]ngineer')))]

q2df.to_csv('./Q2_jobs.csv', index=False)

In [43]:
q2df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   position  100 non-null    object
 1   company   100 non-null    object
 2   city      100 non-null    object
 3   state     100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [44]:
q2df.head(3)

,position,company,city,state
0,Senior Python Developer,"Payne, Roberts and Davis",Stewartbury,AA
1,Energy engineer,Vasquez-Davidson,Christopherville,AA
2,Legal executive,"Jackson, Chambers and Levy",Ericaburgh,AA


In [45]:
q2df.tail(3)

,position,company,city,state
97,Database administrator,Yates-Ferguson,Susan,AE
98,Furniture designer,Ortega-Lawrence,Tiffany,AA
99,Ship broker,"Fuentes, Walls and Castro",Michelleville,AP


In [46]:
q2sample

,position,company,city,state
1,Energy engineer,Vasquez-Davidson,Christopherville,AA
28,Structural engineer,Pierce-Long,Herbertside,AA
32,Broadcast engineer,"Morgan, Butler and Bennett",Loribury,AA
48,"Engineer, broadcasting (operations)",Taylor PLC,Gileston,AA


### Q3 output

In [59]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
requests.packages.urllib3.disable_warnings()

url3='https://www.peakbagger.com/list.aspx?lid=5651'
headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'}
soup= BeautifulSoup(requests.get(url3, headers=headers, verify=False).text, 'html.parser')
table=soup.find_all('table', {'class':"gray"})

mname, region, elev, href, mid,lat, long, tablesum = [], [], [], [], [], [], [], []

for i in table:
    for j in i.find_all('td'):
        tablesum.append(j.text)        
for i in table:
    for j in i.find_all('a'):
        href.append(j.get("href"))
href=href[7:-1:2]      
mname=tablesum[1::8]
region=tablesum[2::8]
elev=tablesum[3::8]

for i in href:
    mid.append(re.findall(r'.+=(\d+)',i)[0])

for i in href:
    temphref='https://www.peakbagger.com/'+i
    tempsoup=BeautifulSoup(requests.get(temphref, headers=headers, verify=False).text, 'lxml')
    temptable=tempsoup.find('table',{'class':"gray"})
    for j in temptable.find_all('td'):
        if templatlong:=re.findall(r"(\d{2,3}.\d{1,6}), (\d{2,3}.\d{1,6})",j.text):
            lat.append(list(templatlong[0])[0])
            long.append(list(templatlong[0])[1])
        
q3df=pd.DataFrame({'name':mname,'region':region,'elev':elev, 'id':mid,'lat':lat, 'long':long})

q3df['elev']=q3df['elev'].astype('int')
q3df['lat']=q3df['lat'].astype('float')
q3df['long']=q3df['long'].astype('float')
q3df['elev_cat']=pd.cut(q3df['elev'],bins=[0,1000,2000,3000,4000],labels=['Cat 1','Cat 2','Cat 3','Cat 4'])

q3sample=pd.pivot_table(q3df,values='id', index=['region'], columns=['elev_cat'], aggfunc='count')
q3df['name']=q3df['name'].astype('str')
q3df['region']=q3df['region'].astype('str')
q3df['id']=q3df['id'].astype('str')
q3df['elev_cat']=q3df['elev_cat'].astype('str')

In [60]:
q3df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      100 non-null    object 
 1   region    100 non-null    object 
 2   elev      100 non-null    int32  
 3   id        100 non-null    object 
 4   lat       100 non-null    float64
 5   long      100 non-null    float64
 6   elev_cat  100 non-null    object 
dtypes: float64(2), int32(1), object(4)
memory usage: 5.2+ KB


In [61]:
q3df.head(3)

,name,region,elev,id,lat,long,elev_cat
0,Fuji-san,Kanto,3776,10882,35.360638,138.727347,Cat 4
1,Kita-dake,Chubu,3192,10866,35.674537,138.238833,Cat 4
2,Hotaka-dake,Chubu,3190,10840,36.289203,137.647986,Cat 4


In [62]:
q3df.tail(3)

,name,region,elev,id,lat,long,elev_cat
97,Ibuki-yama,Western Japan,1377,10879,35.417856,136.406373,Cat 2
98,Kaimon-dake,Western Japan,924,10937,31.179944,130.528397,Cat 1
99,Tsukuba-san,Kanto,877,10844,36.225403,140.106705,Cat 1


In [63]:
q3sample

elev_cat,Cat 1,Cat 2,Cat 3,Cat 4
region,,,,
Chubu,0,4,29,12
Hokkaido,0,5,4,0
Kanto,1,3,8,1
Tohoku,0,11,9,0
Western Japan,1,12,0,0


In [64]:
q3df.to_csv('./Q3_mountains.csv', index=False)

### Q4 output

In [54]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
requests.packages.urllib3.disable_warnings()

url4='https://msc.hkubs.hku.hk/articles/13?op=10&cd=99'
headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'}
msba= BeautifulSoup(requests.get(url4, headers=headers, verify=False).text, 'lxml')

faculty=msba.find('div', {'class':"Facultyitems-block"})
name_, university, url_=[],[],[]
for i in faculty.find_all('b', {'class':"name"}):
    name_.append(i.text)
for i in range(len(name_)):
    name_[i]=name_[i].split(',')[0]
for i in faculty.find_all('p', {'class':"desinfo"}):
    university.append(re.findall('.+?, (.+),.+?',i.text)[0])
for i in faculty.find_all('img'):
    url_.append("https://msc.hkubs.hku.hk/"+i.get('src'))
    
q4df=pd.DataFrame({'name':name_,'university':university,'url':url_})

for i in range(len(name_)):
    r=requests.get(url_[i], headers=headers, verify=False)
    with open('./images/'+name_[i]+'.jpg', 'wb') as handle:
        handle.write(r.content)
        
q4df.to_csv('./Q4_teachers.csv', index=False)


In [57]:
pd.set_option('display.max_colwidth',None)
q4df

,name,university,url
0,Prof. Haipeng SHEN,"The Wharton School of Business, University of Pennsylvania",https://msc.hkubs.hku.hk//uploads/image/202208/912e4fbe417ba7386483c5a63114a859.jpg
1,Dr. Hailiang CHEN,Purdue University,https://msc.hkubs.hku.hk//uploads/image/202205/7bf2456efbfe30faf679e8687b593095.JPG
2,Prof. Xin WANG,Duke University,https://msc.hkubs.hku.hk//uploads/image/202205/f8c2ae64fb58e77abf85ee5554551f57.JPG
3,Prof. Zhenhui Jack JIANG,University of British Columbia,https://msc.hkubs.hku.hk//uploads/image/202205/60ec248db1e94f347d9a768a8fafe269.JPG
4,Dr. Wei ZHANG,Purdue University,https://msc.hkubs.hku.hk//uploads/image/202205/4ffba69f304d9e0b6b87f091e0504495.JPG
5,Prof. Z. Max SHEN,Northwestern University,https://msc.hkubs.hku.hk//uploads/image/202205/3e54b3217a5b21961988b14da9ce33ca.JPG
6,Dr. Zhanrui CAI,The Pennsylvania State University,https://msc.hkubs.hku.hk//uploads/image/202308/37466d97c117c14135605fea682f305b.jpg
7,Prof. Michael C. L. CHAU,Arizona University,https://msc.hkubs.hku.hk//uploads/image/202205/ac5d364416b5bb9a1714038c2c4d5276.jpg
8,Dr. Chao DING,University of Florida,https://msc.hkubs.hku.hk//uploads/image/202205/caadde230670b19f303e4d280934e054.JPG
9,Dr. Xing HU,"Stern School of Business, New York University",https://msc.hkubs.hku.hk//uploads/image/202205/b1afbc52a0554880ed85aa5e60cff5fa.JPG


In [58]:
q4df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        25 non-null     object
 1   university  25 non-null     object
 2   url         25 non-null     object
dtypes: object(3)
memory usage: 732.0+ bytes
